# Generate bounding boxes from place names

Purpose: This script uses pandas and the OpenStreetMap Nominatim API to geocode a list of place names in a CSV file and return bounding box coordinates for each place.

## 1. Import modules

In [ ]:
import pandas as pd
import requests

## 2. Define the function to get the coordinates

This function scans the OpenStreetMap API and will match the place names to the bounding boxes.  

The [GeoBTAA input guidelines](https://geobtaa.github.io/metadata/input-guidelines/#bounding-box) specifies coordinates in this order: `West, South, East, North`. When we return the coordinates, we reorder them (line #13)

In [ ]:
# Define function to geocode a place name using Nominatim API
def geocode_place_name(place_name):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": place_name,
        "format": "jsonv2"
    }
    response = requests.get(url, params=params)
    if response.ok:
        data = response.json()
        if len(data) > 0 and "boundingbox" in data[0]:
            bbox = data[0]["boundingbox"]
            return [bbox[2], bbox[0], bbox[3], bbox[1]]
    return None

## 3. Load the CSV file into a pandas dataframe

In [ ]:
df = pd.read_csv("place-names_input.csv")

# print the dataframe
df

Our input CSV contains three columns:

* Title
* Spatial Coverage (place name text)
* Bounding Box - empty

## 4. Apply the geocode_place_name function to the "place" column

Depending upon how many items you are querying, this step may take some time to run.

In [ ]:
df["Bounding Box"] = df["Spatial Coverage"].apply(geocode_place_name)

# print the dataframe
df

## 5. Write to CSV

The bounding box field is now populated with 4 coordinates. To save this as a CSV, run the next cell.

In [ ]:
# Write the results to a new CSV file
df.to_csv("place-names_output.csv", index=False)